In [ ]:
# AUTO click
from pynput.mouse import Controller,Button
import time

mouse = Controller()

while True:
    mouse.click(Button.left,1)
    print('clicked')
    time.sleep(10)

In [2]:
import cv2,dlib, random,os, csv, time
import numpy as np
from wear_mask.FacialMaskDataSet import FacialMaskDataSet
from wear_mask import MTCNN
from tool.FileFunction import FileFunction
# jupyter notebook   --NotebookApp.allow_origin='https://colab.research.google.com'   --port=8888   --NotebookApp.port_retries=0

Instructions for updating:
non-resource variables are not supported in the long term


# Demo

## Demo wear mask camera

In [ ]:
facial_mask_dataset = FacialMaskDataSet()
file_function =  FileFunction()
#----initialization of MTCNN model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(os.path.join(os.getcwd(),'./models/shape_predictor_68_face_landmarks.dat'))
# Read mask pictures
maskImagePaths = file_function.getPath(os.path.join(os.getcwd(),"mask_image"))
numberOfMask = len(maskImagePaths)
if numberOfMask <=0:
    print("there are no mask in ")
offsetRandomMask = random.randint(0, numberOfMask - 1)
# Open video
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
timer = time.time()
while(True):
            if (time.time() - timer) >= 2:
                print (time.time()-timer)
                timer = time.time()
                offsetRandomMask = random.randint(0, numberOfMask - 1)
            # capture frame by frame
            ret, frame = cap.read()
            xMin, xMax, yMin, yMax, size, landmarks =facial_mask_dataset.findMouthCoordinate(frame,detector,predictor)
            if size is not None:
                # If can find all land mark use Homography to overlay mask
                landmarks = np.array([[point.x,point.y] for point in landmarks.parts()])
                if (len(landmarks) != 0) and (landmarks is not None) and (landmarks > 0).all():
                    pathRandomMask = maskImagePaths[offsetRandomMask]
                    imageRandomMask = cv2.imread(pathRandomMask, cv2.IMREAD_UNCHANGED)
                    coordinatePointsFace = [[point[0], point[1]] for point in landmarks[1:16:1]]
                    coordinatePointsFace.append([landmarks[29][0],landmarks[29][1]])
                    nameMask = pathRandomMask.split("\\")[-1].split(".")[0]
                    pathAnnotation = os.path.join(os.path.join(os.getcwd(), "mask_annotation"),nameMask+".csv")
                    with open(pathAnnotation) as annotationFile:
                        csvReader = csv.reader(annotationFile)
                        coordinatePointsMask = []
                        for row in csvReader:
                            try:
                                coordinatePointsMask.append([float(row[1]), float(row[2])])
                            except ValueError:
                                continue
                    coordinatePointsFace = np.array(coordinatePointsFace, dtype = np.float)
                    coordinatePointsMask = np.array(coordinatePointsMask, dtype=np.float)
                    frame = facial_mask_dataset.attachMaskToFaceHomoGraphy(frame, imageRandomMask, coordinatePointsMask, coordinatePointsFace )
            cv2.imshow("frame", frame)
            if(cv2.waitKey(1) == ord("q")):
                break
cap.release()
cv2.destroyAllWindows()

# Face recognize

In [ ]:
from train.FaceNet import FaceNetModel,call_instance_FaceNet_with_custom, call_instance_FaceNet_without_custom, call_instance_FaceNet_with_last_isDense,convert_train_model_to_embedding
from PIL import Image
import numpy as np
import tensorflow as tf
from train.Classify import Classify
from tool.FormatFunction import FormatFunction
from tool.FileFunction import FileFunction
from tool.GlobalValue import GlobalValue



global_value = GlobalValue(image_size=[110,110], batch_size = 512, shuffle_size = 1000, ratio_train = 0.8, ratio_test = 0.1, ratio_valid = 0.1, epochs = 40, small_epochs = 50,
                           image_each_class = 15)
format_function = FormatFunction(global_value)
model_path = os.path.join(os.getcwd(),"")
reload_model  = call_instance_FaceNet_with_last_isDense(global_value.IMAGE_SIZE,10575)
reload_model.load_weights(model_path)
embedding_model = convert_train_model_to_embedding(reload_model)


In [3]:
import numpy as np
vector = [1,2,3]
vector = vector/np.linalg.norm(vector)
print(vector)

print(np.linalg.norm(vector))

[0.26726124 0.53452248 0.80178373]
1.0


In [5]:
def myFunc():
    return 1,2,3

a,_,_ = myFunc()
print(a)

1
